# Web Programming in Java

## JDBC

* Follow the instructions to install the database H2;
* in the project `enrollSys`, create a package to take care of database connections: `ca.edu.collegeX.enrollSys.jdbc`
* JDBC (Java DataBase Connectivity): set of well-defined interfaces inside the package `java.sql`; 
* driver JDBC: concrete classes making a link between API JDBC and the database; 
* in the package `ca.edu.collegeX.enrollSys.jdbc`, create a class named `ConnectionFactory` with the code: 

```java
package ca.edu.collegeX.enrollSys.jdbc;

public class ConnectionFactory {
	public Connection getConnection(){
		System.out.println("Connecting to the database");
		try {
		// the line below is not necessary anymore from JDBC 4 (Java 6)
		//Class.forName("org.h2.Driver");
		return DriverManager.getConnection("jdbc:h2:tcp://localhost/~progweb","admin","admin");
		} catch (Exception e) {
			throw new RuntimeException(e);
		}
	}
}
```

* add the imports needed (`java.sql.Connection` and `java.sql.DriverManager`) using the shortcut `Ctrl + 1`;
* add the driver of H2 (a JAR file) containing the implementation JDBC of H2 to the classpath of the project. The driver of H2 is called `h2-XXXX.jar` (where XXXX is the version) and is inside the folder `h2`, folder `bin`. Follow the steps: right click over the project, option `Build Path`, then `Configure Build Path`, next `Libraries` and `Add External JARs`. 
* Test the connection with the following code (add exception handling with `try-catch` using `Ctrl + 1`): 

```java
// testing connection
Connection connection = new ConnectionFactory().getConnection();
System.out.println("Connection done");
connection.close(); 
```

* make and test the connection; 
* always handle exceptions and always remember to close connections to the database;
* the drivers can be downloaded from manufacturer's website; 
* why a connection factory: 
    * only one place to modify when/if needed; 
    * possibility to create a pool of connections; 
    * it follows design patterns about encapsulating the construction of complex objects (see Go4 book);
    
    * in the project `enrollSys`, create a package responsible for Data Access Objects (DAO): `ca.edu.collegeX.enrollSys.dao`
* in the package `ca.edu.collegeX.enrollSys.dao`, create a Javabean named `StudentDAO`; 
* remember to import the class `java.sql` (**careful**: it’s not the class `com.sql`); 
* add the connection with the database in the constructor of `StudentDAO`:

```java
public class StudentDAO { 
    private Connection connection; 

    public StudentDAO() { 
        this.connection = new ConnectionFactory().getConnection(); 
    }
}
```

* implement the insertion:  

```java
public void insert(Student student) {
    String sql = "insert into students (name,email,address) values (?,?,?)";

    try { // prepared statement for insertion
        PreparedStatement stmt = connection.prepareStatement(sql);

        // it sets the values
        stmt.setString(1, student.getName());
        stmt.setString(2, student.getEmail());
        stmt.setString(3, student.getAddress());

        // it runs
        stmt.execute();

        // it closes statement
        stmt.close();
    } catch (SQLException e) {
        throw new RuntimeException(e);
    }
}
```

* modify the class `CreateStudent`, by adding a code to test the insertion:

```java
// recording a student
StudentDAO dao = new StudentDAO();
dao.insert(student);
System.out.println("Recorded!");
```

* problems with SQL:

```java
String sql = "insert into students (name, email, address) values ('" + name + "','" + 
                email + "'+'" + address + "')";
```

* difficult to read; 
* likely to have errors; 
* prejudice against Joana D'Arc; 
* SQL Injection: technique used to attack data-driven applications. **Example**:

<img src="images/exploits_of_a_mom.png">
<center><b>Source</b>: http://xkcd.com/327/ </center>

* sanitize inputs for database: input handling (special characters, etc); 
* more generic and more secure:

```java
String sql = "insert into students (name,email,address) values (?,?,?)";
```

* `PreparedStatement`: interface to execute clauses, it sanitizes inputs for database;    

* Implement the deletion of records:

```java
public void remove(Student student) {
    try {
        PreparedStatement stmt = connection.prepareStatement("delete from students where id=?");
        stmt.setLong(1, student.getId());
        stmt.execute();
        stmt.close();
    } catch (SQLException e) {
        throw new RuntimeException(e);
    }
}
```

* Modify the class `CreateStudent`, adding a code to test the insertion method (remember to set a value for the `id`):

```java
// removing record
StudentDAO dao = new StudentDAO();
dao.remove(student);
System.out.println("Removed!");
```

* implement a method for updating:

```java
public void update(Student student) {
    String sql = "update students set name=?, email=?, address=? where id=?";

    try {
        PreparedStatement stmt = connection.prepareStatement(sql);
        stmt.setString(1, student.getName());
        stmt.setString(2, student.getEmail());
        stmt.setString(3, student.getAddress());
        stmt.setLong(4, student.getId());
        stmt.execute();
        stmt.close();
    } catch (SQLException e) {
        throw new RuntimeException(e);
    }
}
```

* modify the class `CreateStudent`, adding a code to test the method for updating (remember to provide an `id`):

```java
// updating a record
StudentDAO dao = new StudentDAO();
dao.update(student);
System.out.println("Updated!");
```

* implement a method to list all records:

```java
public List<Student> getList() {
    List<Student> students = new ArrayList<Student>(); // import java.util for this

    PreparedStatement stmt;
    try {
        stmt = this.connection.prepareStatement("select * from students order by name");

        ResultSet rs = stmt.executeQuery(); // import java.sql

        while (rs.next()) {
            Student student = new Student();
            student.setId(rs.getLong("id"));
            student.setName(rs.getString("name"));
            student.setEmail(rs.getString("email"));
            student.setAddress(rs.getString("address"));

            students.add(student);
        }
        rs.close();
        stmt.close();
    } catch (SQLException e) {
        throw new RuntimeException(e);
    }
    return students;
}
```

* modify the class `CreateStudent`, adding a code to test the method `getList()`:

```java
// listing records
StudentDAO dao = new StudentDAO();
List<Student> students = dao.getList();
for ( Student student1 : students){
        System.out.println("Name: " + student1.getName() + " Email: " + student1.getEmail() + " Address: " + student1.getAddress());
}
```

* Use the methods we just created to insert, remove, update and list in order to make just tests with the database through the Java application. Using the SQL command select, verify in the database if the changes are there.

## Exercise
Make a CRUD to store your phone contacts:
* Create a database named `contactsdb` with a table `contacts` containing the fields `id`, `name`, `email`, `address`;
* Create a Java application and implement the operations of insertion, updating, deletion, searching by name and listing all contacts.